## Ejercicio: Visualización y análisis de predicciones en España

En el ejercicio 2 hacíamos un vistazo inicial y respondíamos a ciertas preguntas sobre el dataset. En este ejercicio se utilizarán gráficos para responder a las siguientes preguntas:

1. Como se distribuyen las ventas realizadas en:
    - Cada país
    - Cada mes y año
    - Cada marca
2. Cual es la tendencia y estacionalidad de:
    - Todas las ventas del país con menos ventas
    - La marca con más ventas
3. Cuales son las predicciones hechas en España y como de
buenas son.

In [525]:
import pandas as pd

df = pd.read_csv("datasets/datos_ejercicio_ventas.csv")
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


El dataset cuenta con los siguientes campos:
- COUNTRY: País en el que se realiza la operación (Ejemplo: Portugal)

- SUBBRAND: Producto del que se tiene el dato (Ejemplo: Lipton (L3))

- YEAR and MONTH: Cada par nos indica un instante en el tiempo (Ejemplo: 2023-12) 

- SCENARIO: Tipo de dato (Predicción o actual)

- FORECAST: En caso de existir nos dice el mes en el que se hace la predicción (AI_P02F se refiere a predicciones hechas en enero)

- FORECAST_YEAR: Año en el que se realiza la predicción (Para este dataset solo se han hecho predicciones en 2023)

- AMOUNT: Cantidad (Estimada o real en función de si es predicción o actual)


In [526]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18666 entries, 0 to 18665
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   COUNTRY        18666 non-null  object 
 1   SUBBRAND       18666 non-null  object 
 2   YEAR           18666 non-null  int64  
 3   MONTH          18666 non-null  int64  
 4   SCENARIO       18666 non-null  object 
 5   FORECAST       17766 non-null  object 
 6   FORECAST_YEAR  17766 non-null  float64
 7   AMOUNT         18666 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.1+ MB


Existen valores NA que hacen referencia a que no se tienen datos de predicción cuando se trata de un actual.

In [527]:
na_df = pd.DataFrame({'NA_Counts': df.isna().sum()})
na_df

,NA_Counts
COUNTRY,0
SUBBRAND,0
YEAR,0
MONTH,0
SCENARIO,0
FORECAST,900
FORECAST_YEAR,900
AMOUNT,0


In [528]:
scenario_count = df.loc[:, "SCENARIO"].value_counts().to_frame()
scenario_count.index.name = None
scenario_count

,count
AI_forecast,17766
actual,900


Podemos visualizar el porcentaje de cada tipo

In [529]:
import plotly.express as px
fig = px.pie(
    scenario_count,
    values="count",
    names=scenario_count.index,
    title='Predicciones vs Actuals'
)
fig.show()

Dividimos en 2 el dataset, uno que contiene los actuals y otro que contiene las predicciones:

In [530]:
actuals_mask = df.loc[:, "SCENARIO"] == "actual"
df_actuals = df.loc[actuals_mask, :].reset_index(drop=True)
df_actuals.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Pepsi Max (L3),2023,10,actual,NaN,NaN,1.885949e+05
1,Portugal,7up (L3),2023,3,actual,NaN,NaN,2.934971e+05
2,Portugal,7up (L3),2023,10,actual,NaN,NaN,3.484466e+05
3,Great Britain,7up Free (L3),2023,10,actual,NaN,NaN,1.172553e+06
4,Norway,Pepsi Regular (L3),2023,10,actual,NaN,NaN,3.784859e+04


Del cual podemos eliminar las columnas de predicción (`AMOUNT` es el valor real) y la columna `SCENARIO` al tratarse este subconjunto solo de actuals

In [531]:
df_actuals = df_actuals.dropna(axis=1, how="all")
df_actuals = df_actuals.drop(columns="SCENARIO")
df_actuals.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,AMOUNT
0,Portugal,Pepsi Max (L3),2023,10,1.885949e+05
1,Portugal,7up (L3),2023,3,2.934971e+05
2,Portugal,7up (L3),2023,10,3.484466e+05
3,Great Britain,7up Free (L3),2023,10,1.172553e+06
4,Norway,Pepsi Regular (L3),2023,10,3.784859e+04


Para las predicciones la máscara es inversa:

In [532]:
df_forecasts = df.loc[~actuals_mask, :]
df_forecasts.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


De nuevo la columna SCENARIO no nos aporta información (con el nombre de la variable nos basta)

In [533]:
df_forecasts = df_forecasts.drop(columns="SCENARIO")
df_forecasts.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_P03F,2023.0,396176.120491


Otro formato útil es convertir las fechas a tipo datetime, sustituyendo `MONTH` y `YEAR` por `DATE`.

In [534]:
df_actuals["DATE"] = pd.to_datetime(df_actuals[['YEAR', 'MONTH']].assign(DAY=1))
df_actuals = df_actuals.drop(columns=["YEAR", "MONTH"])
df_actuals.head()

,COUNTRY,SUBBRAND,AMOUNT,DATE
0,Portugal,Pepsi Max (L3),1.885949e+05,2023-10-01
1,Portugal,7up (L3),2.934971e+05,2023-03-01
2,Portugal,7up (L3),3.484466e+05,2023-10-01
3,Great Britain,7up Free (L3),1.172553e+06,2023-10-01
4,Norway,Pepsi Regular (L3),3.784859e+04,2023-10-01


In [535]:
df_forecasts["DATE"] = pd.to_datetime(df_forecasts[['YEAR', 'MONTH']].assign(DAY=1))
df_forecasts = df_forecasts.drop(columns=["YEAR", "MONTH"]).reset_index(drop=True)
df_forecasts.head()

,COUNTRY,SUBBRAND,FORECAST,FORECAST_YEAR,AMOUNT,DATE
0,Portugal,Lipton (L3),AI_P02F,2023.0,754356.237194,2023-12-01
1,Great Britain,Lipton (L3),AI_P10F,2023.0,560030.558029,2023-12-01
2,Spain,Pepsi Max (L3),AI_P09F,2023.0,88501.980847,2023-12-01
3,Great Britain,7up (L3),AI_P10F,2023.0,363224.511516,2024-12-01
4,Hungary,Lipton (L3),AI_P03F,2023.0,396176.120491,2023-09-01


Para el caso de forecasts también se convierte a datetime cuando se ha hecho la predicción combinando las columnas 

In [536]:
forecast_date = df_forecasts["FORECAST"].unique()
print(forecast_date)
print(f"Número de valores posibles = {len(forecast_date)}")

['AI_P02F' 'AI_P10F' 'AI_P09F' 'AI_P03F' 'AI_PF' 'AI_P11F' 'AI_P06F'
 'AI_P05F' 'AI_P07F' 'AI_P12F' 'AI_P08F' 'AI_P04F']
Número de valores posibles = 12


Se tiene en cuenta lo visto en clase, esto es que `AI_P0NF` se refiere a predicciones hechas en el mes anterior al mes `N`. Y `AI_PF` se refiere predicciones hechas al final de el año (Diciembre). Le podemos pedir a la IA que nos genere una función que cree el mapeado a una nueva columna `FORECAST_MONTH` utilizando expresiones regulares.

In [537]:
import re
import numpy as np

# Función para extraer el número y restarle 1, o devolver 12 si es "AI_PF"
def extraer_valor(cadena):
    # Verificar si la cadena es "AI_PF"
    if cadena == 'AI_PF':
        return 12
    # Usar expresión regular para extraer el número entre "P" y "F"
    match = re.search(r'P(\d+)F', cadena)
    if match:
        return int(match.group(1)) - 1
    return None  # En caso de que no se encuentre un patrón válido

np.sort(df_forecasts['FORECAST'].apply(extraer_valor).unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

Y aplicamos la función para sustituir `FORECAST` y `FORECAST_YEAR` por `FORECAST_DATE` (pasando por `FORECAST_MONTH`)

In [538]:
# Columna que indica el mes en el que se hace la predicción
df_forecasts['FORECAST_MONTH'] = df_forecasts['FORECAST'].apply(extraer_valor)
df_forecasts["FORECAST_DATE"] = pd.to_datetime(
    df_forecasts[["FORECAST_MONTH", "FORECAST_YEAR"]]
    .assign(DAY=1)
    .rename(columns={"FORECAST_MONTH": "MONTH", "FORECAST_YEAR": "YEAR"}) # Necesario (pandas es inconsistente aquí)
)
df_forecasts = df_forecasts.drop(columns=["FORECAST_MONTH", "FORECAST_YEAR", "FORECAST"])
df_forecasts.head()

,COUNTRY,SUBBRAND,AMOUNT,DATE,FORECAST_DATE
0,Portugal,Lipton (L3),754356.237194,2023-12-01,2023-01-01
1,Great Britain,Lipton (L3),560030.558029,2023-12-01,2023-09-01
2,Spain,Pepsi Max (L3),88501.980847,2023-12-01,2023-08-01
3,Great Britain,7up (L3),363224.511516,2024-12-01,2023-09-01
4,Hungary,Lipton (L3),396176.120491,2023-09-01,2023-02-01


### 1.1. Distribución de ventas por país:

Al referirse el enunciado a ventas se entiende que se refiere a ventas reales. Esto es nuestro dataframe `df_actuals`.

In [539]:
df_actuals.head()

,COUNTRY,SUBBRAND,AMOUNT,DATE
0,Portugal,Pepsi Max (L3),1.885949e+05,2023-10-01
1,Portugal,7up (L3),2.934971e+05,2023-03-01
2,Portugal,7up (L3),3.484466e+05,2023-10-01
3,Great Britain,7up Free (L3),1.172553e+06,2023-10-01
4,Norway,Pepsi Regular (L3),3.784859e+04,2023-10-01


Podemos agrupar los datos por país y ordenar para obtener un ranking de país en función de su `AMOUNT`.

In [540]:
country_distribution = (
    df_actuals
    .groupby('COUNTRY')['AMOUNT']
    .sum()
    .to_frame()
    .sort_values("AMOUNT", ascending=False)
)
country_distribution

,AMOUNT
COUNTRY,
Great Britain,3.347786e+08
Netherlands,6.395943e+07
Denmark,5.659668e+07
Norway,5.121406e+07
Italy,4.345404e+07
Hungary,4.153991e+07
Czech,3.535164e+07
Portugal,3.488807e+07
Spain,8.131266e+06


In [542]:
fig = px.bar(country_distribution, 
             x=country_distribution.index, y='AMOUNT', 
             title='Sales Volume by Country')

fig.show()